In [ ]:
import tensorflow as tf
import tensorflow_io as tfio
from IPython.display import Audio
import glob

In [ ]:
audio = tfio.audio.AudioIOTensor("x1/blues00003_x1.wav")
print(audio)

In [ ]:
audio_files = glob.glob('x1/*.wav')

In [45]:
audio_files

['x1/blues00000_x1.wav',
 'x1/blues00001_x1.wav',
 'x1/blues00002_x1.wav',
 'x1/blues00003_x1.wav',
 'x1/blues00004_x1.wav',
 'x1/blues00005_x1.wav',
 'x1/blues00006_x1.wav',
 'x1/blues00007_x1.wav',
 'x1/blues00008_x1.wav',
 'x1/blues00009_x1.wav',
 'x1/blues00010_x1.wav',
 'x1/blues00011_x1.wav',
 'x1/blues00012_x1.wav',
 'x1/blues00013_x1.wav',
 'x1/blues00014_x1.wav',
 'x1/blues00015_x1.wav',
 'x1/blues00016_x1.wav',
 'x1/blues00017_x1.wav',
 'x1/blues00018_x1.wav',
 'x1/blues00019_x1.wav',
 'x1/blues00020_x1.wav',
 'x1/blues00021_x1.wav',
 'x1/blues00022_x1.wav',
 'x1/blues00023_x1.wav',
 'x1/blues00024_x1.wav',
 'x1/blues00025_x1.wav',
 'x1/blues00026_x1.wav',
 'x1/blues00027_x1.wav',
 'x1/blues00028_x1.wav',
 'x1/blues00029_x1.wav',
 'x1/blues00030_x1.wav',
 'x1/blues00031_x1.wav',
 'x1/blues00032_x1.wav',
 'x1/blues00033_x1.wav',
 'x1/blues00034_x1.wav',
 'x1/blues00035_x1.wav',
 'x1/blues00036_x1.wav',
 'x1/blues00037_x1.wav',
 'x1/blues00038_x1.wav',
 'x1/blues00039_x1.wav',


In [50]:
def load_and_process_audio(file_path):
    audio = tfio.audio.AudioIOTensor(file_path, dtype=tf.float32)  # Adjust dtype as needed
    # Add any additional processing steps here
    label = 0  # Replace this with the actual label associated with your data
    return audio, label


In [51]:
dataset = tf.data.Dataset.from_tensor_slices(audio_files)

In [52]:
dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

In [53]:
dataset = dataset.map(load_and_process_audio)

TypeError: Unsupported return value from function passed to Dataset.map(): (<AudioIOTensor: shape=Tensor("AudioIOTensor/IO>AudioReadableSpec:0", shape=(2,), dtype=int64), dtype=<dtype: 'float32'>, rate=Tensor("AudioIOTensor/IO>AudioReadableSpec:2", shape=(), dtype=int32)>, 0).

In [ ]:
audio_slice = audio[100:]
audio_tensor = tf.squeeze(audio_slice, axis=[-1])

In [ ]:
Audio(audio_tensor.numpy(), rate=audio.rate.numpy())

In [ ]:
print(audio_tensor)

In [ ]:
import matplotlib.pyplot as plt


tensor = tf.cast(audio_tensor, tf.float32) / 32768.0

plt.figure()
plt.plot(tensor.numpy())

In [ ]:
position = tfio.audio.trim(tensor, axis=0, epsilon=0.1)
print(position)

start = position[0]
stop = position[1]
print(start, stop)

processed = tensor[start:stop]

plt.figure()
plt.plot(processed.numpy())

In [ ]:
fade = tfio.audio.fade(
    processed, fade_in=1000, fade_out=2000, mode="logarithmic")

plt.figure()
plt.plot(fade.numpy())

In [ ]:
# Convert to spectrogram
spectrogram = tfio.audio.spectrogram(
    fade, nfft=512, window=512, stride=256)

plt.figure()
plt.imshow(tf.math.log(spectrogram).numpy())

In [ ]:
# Convert to mel-spectrogram
mel_spectrogram = tfio.audio.melscale(
    spectrogram, rate=16000, mels=128, fmin=0, fmax=8000)


plt.figure()
plt.imshow(tf.math.log(mel_spectrogram).numpy())

# Convert to db scale mel-spectrogram
dbscale_mel_spectrogram = tfio.audio.dbscale(
    mel_spectrogram, top_db=80)

plt.figure()
plt.imshow(dbscale_mel_spectrogram.numpy())